In [120]:
import pandas as pd
import numpy as np

import os
import sys
sys.path.append('..')

from pathlib import Path
import datetime as dt

import wget
import requests
from zipfile import ZipFile

## Data Collection

### Quantidade de Ações

In [219]:
def download_cvm_data(
    data_type,
    initial_year=2011, 
    last_year=2022, 
    save_folder='.'
):

    download_path = os.path.join(save_folder, 'downloaded')
    os.makedirs(download_path, exist_ok=True)

    if data_type=='fre':
        BASE_URL = 'https://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/FRE/DADOS/'
    elif data_type=='itr':
        BASE_URL = 'http://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/ITR/DADOS/'
    if data_type=='dfp':
        BASE_URL = 'https://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/DFP/DADOS/'

    filename_list = []
    for year in range(initial_year, last_year + 1):
        filename_list.append(f'{data_type}_cia_aberta_{year}.zip')

    for filename in filename_list:
        try:
            download_path = os.path.join(download_path, filename)
            
            if not Path(download_path).exists():
                wget.download(
                    BASE_URL + filename, 
                    out=download_path
                )
                
        except Exception as e:
            print(f'The file of year {year} was not found.')
            print('Exception Message:', e, end='\n\n')


In [221]:
itr_save_folder = os.path.join('..', 'cvm_data', 'itr')
fre_save_folder = os.path.join('..', 'cvm_data', 'fre')
dfp_save_folder = os.path.join('..', 'cvm_data', 'dfp')

download_cvm_data('itr', 2011, 2022, save_folder=itr_save_folder)
download_cvm_data('dfp', 2011, 2022, save_folder=dfp_save_folder)
download_cvm_data('fre', 2011, 2022, save_folder=fre_save_folder)

In [222]:
def extract_all_files(folder_path):
    filename_list = os.listdir(folder_path)

    extract_path = os.path.join(Path(folder_path).parent, 'extracted')
    for filename in filename_list:
        ZipFile(
            os.path.join(folder_path, filename), 
            'r'
        ).extractall(extract_path)


extract_all_files(os.path.join(itr_save_folder, 'downloaded'))
extract_all_files(os.path.join(fre_save_folder, 'downloaded'))
extract_all_files(os.path.join(dfp_save_folder, 'downloaded'))

### ITR

In [81]:
# DOWNLOAD_PATH = os.path.join('..', 'cvm_data', 'downloaded')
# os.makedirs(DOWNLOAD_PATH, exist_ok=True)

# BASE_URL = 'http://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/ITR/DADOS/'

# filename_list = []
# for year in range(2011, 2022):
#     filename_list.append(f'itr_cia_aberta_{year}.zip')


# for filename in filename_list:
#     try:
#         save_path = os.path.join(DOWNLOAD_PATH, filename)
        
#         if not Path(save_path).exists():
#             wget.download(
#                 BASE_URL + filename, 
#                 out=save_path
#             )
            
#     except Exception as e:
#         print(f'The file of year {year} was not found.')
#         print('Exception Message:', e, end='\n\n')

In [32]:
# for filename in filename_list:
#     ZipFile(os.path.join(DOWNLOAD_PATH, filename), 'r').extractall('..\cvm_data\extracted')

In [ ]:
for year in range(initial_year, last_year+1):

In [229]:
def consolidate_years(folder_path, data_type, document_names, initial_year, last_year):

    for document_name in document_names:
        df_file = pd.DataFrame()
        for year in range(initial_year, last_year+1):
            df_file = pd.concat([
                df_file,
                pd.read_csv(
                    os.path.join(
                        folder_path,
                        f'{data_type}_cia_aberta_{document_name}_{year}.csv'
                    ),
                    sep=';',
                    decimal=',',
                    encoding='ISO-8859-1'
                )
            ])

        save_path = os.path.join(
            Path(folder_path).parent, 
            'consolidated',
            f'{data_type}_{document_name}_2011_2021.csv'
        )
        os.makedirs(Path(save_path).parent, exist_ok=True)

        df_file.to_csv(save_path)


document_names = ['DRE_con', 'BPP_con']
consolidate_years(
    os.path.join(itr_save_folder, 'extracted'),
    'itr', 
    document_names, 
    2011, 2021
)

document_names = ['DRE_con', 'BPP_con']
consolidate_years(
    os.path.join(dfp_save_folder, 'extracted'),
    'dfp', 
    document_names, 
    2011, 2021
)

document_names = ['capital_social']
consolidate_years(
    os.path.join(fre_save_folder, 'extracted'),
    'fre', 
    document_names, 
    2011, 2021
)

## Data Processing

In [274]:
COD_CVM = 9512
CNPJ = '33.000.167/0001-01'

In [275]:
    
def preprocess_dre(codigo_cvm):

    df_dfp = pd.read_csv('..\cvm_data\dfp\consolidated\dfp_DRE_con_2011_2021.csv', index_col=0)
    df_dfp.insert(11, 'TIPO_PERIODO','ANUAL')
    df_itr = pd.read_csv('..\cvm_data\itr\consolidated\itr_DRE_con_2011_2021.csv', index_col=0)
    df_itr.insert(11, 'TIPO_PERIODO','TRIMESTRAL')

    df = pd.concat([df_dfp, df_itr])

    df = df[df['CD_CVM'] == codigo_cvm]
    df = df[df['ORDEM_EXERC'] == 'ÚLTIMO']

    df['DT_REFER'] = pd.to_datetime(df['DT_REFER'])
    df['DT_INI_EXERC'] = pd.to_datetime(df['DT_INI_EXERC'])
    df['DT_FIM_EXERC'] = pd.to_datetime(df['DT_FIM_EXERC'])

    # IGNORA SEMESTRAL
    period_days = (df['DT_FIM_EXERC'] - df['DT_INI_EXERC']).dt.days
    df = df[(period_days<100) | (period_days>360)]

    return df

df = preprocess_dre(COD_CVM)
df

C:\Users\edyalenquer\AppData\Roaming\Python\Python37\site-packages\numpy\lib\arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,CNPJ_CIA,DT_REFER,VERSAO,DENOM_CIA,CD_CVM,GRUPO_DFP,MOEDA,ESCALA_MOEDA,ORDEM_EXERC,DT_INI_EXERC,DT_FIM_EXERC,TIPO_PERIODO,CD_CONTA,DS_CONTA,VL_CONTA,ST_CONTA_FIXA
15253,33.000.167/0001-01,2011-12-31,2,PETROLEO BRASILEIRO S.A. PETROBRAS,9512,DF Consolidado - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2011-01-01,2011-12-31,ANUAL,3.01,Receita de Venda de Bens e/ou Serviços,2.441761e+08,S
15255,33.000.167/0001-01,2011-12-31,2,PETROLEO BRASILEIRO S.A. PETROBRAS,9512,DF Consolidado - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2011-01-01,2011-12-31,ANUAL,3.02,Custo dos Bens e/ou Serviços Vendidos,-1.669393e+08,S
15257,33.000.167/0001-01,2011-12-31,2,PETROLEO BRASILEIRO S.A. PETROBRAS,9512,DF Consolidado - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2011-01-01,2011-12-31,ANUAL,3.03,Resultado Bruto,7.723688e+07,S
15259,33.000.167/0001-01,2011-12-31,2,PETROLEO BRASILEIRO S.A. PETROBRAS,9512,DF Consolidado - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2011-01-01,2011-12-31,ANUAL,3.04,Despesas/Receitas Operacionais,-3.300816e+07,S
15261,33.000.167/0001-01,2011-12-31,2,PETROLEO BRASILEIRO S.A. PETROBRAS,9512,DF Consolidado - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2011-01-01,2011-12-31,ANUAL,3.04.01,Despesas com Vendas,-8.950043e+06,S
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94099,33.000.167/0001-01,2021-09-30,1,PETROLEO BRASILEIRO S.A. PETROBRAS,9512,DF Consolidado - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2021-07-01,2021-09-30,TRIMESTRAL,3.99.01.01,ON,2.390000e+00,N
94103,33.000.167/0001-01,2021-09-30,1,PETROLEO BRASILEIRO S.A. PETROBRAS,9512,DF Consolidado - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2021-07-01,2021-09-30,TRIMESTRAL,3.99.01.02,PN,2.390000e+00,N
94107,33.000.167/0001-01,2021-09-30,1,PETROLEO BRASILEIRO S.A. PETROBRAS,9512,DF Consolidado - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2021-07-01,2021-09-30,TRIMESTRAL,3.99.02,Lucro Diluído por Ação,0.000000e+00,N
94111,33.000.167/0001-01,2021-09-30,1,PETROLEO BRASILEIRO S.A. PETROBRAS,9512,DF Consolidado - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2021-07-01,2021-09-30,TRIMESTRAL,3.99.02.01,ON,2.390000e+00,N


### Trata Lucro Líquido

In [276]:
def get_lucro_liquido_trimestral(df):
    df = df.copy()
    
    first_year = df['DT_REFER'].dt.year.min
    last_year = df['DT_REFER'].dt.year.max

    df = df.loc[
        (df['DS_CONTA'].str.contains('Lucro')) &
        (df['DS_CONTA'].str.contains('Prejuízo')) &
        (df['DS_CONTA'].str.contains('Consolidado'))
    ].copy()
    for year in range(2011, 2022):

        df_temp = df[(df['DT_REFER'].dt.year == year)]

        trimestral_sum = df_temp[df_temp['TIPO_PERIODO']=='TRIMESTRAL']['VL_CONTA'].sum()

        anual_index = df_temp[df_temp['TIPO_PERIODO']=='ANUAL'].index[0]

        df.loc[anual_index, 'VL_CONTA'] -= trimestral_sum
        df.loc[anual_index, 'TIPO_PERIODO'] = 'TRIMESTRAL'

        

    return df

df_petr_lucro_liquido = get_lucro_liquido_trimestral(df)
df_petr_lucro_liquido = df_petr_lucro_liquido[['CNPJ_CIA', 'DT_REFER', 'VL_CONTA']]
df_petr_lucro_liquido.columns = ['cnpj', 'dt_refer', 'value']
df_petr_lucro_liquido.insert(2, 'description', 'lucro_liquido')
df_petr_lucro_liquido

,cnpj,dt_refer,description,value
15315,33.000.167/0001-01,2011-12-31,lucro_liquido,5230817.0
15115,33.000.167/0001-01,2012-12-31,lucro_liquido,7766988.0
15003,33.000.167/0001-01,2013-12-31,lucro_liquido,5950597.0
14742,33.000.167/0001-01,2014-12-31,lucro_liquido,-26961740.0
14496,33.000.167/0001-01,2015-12-31,lucro_liquido,-35582000.0
14435,33.000.167/0001-01,2016-12-31,lucro_liquido,2760000.0
14135,33.000.167/0001-01,2017-12-31,lucro_liquido,-5372000.0
14109,33.000.167/0001-01,2018-12-31,lucro_liquido,2978000.0
16367,33.000.167/0001-01,2019-12-31,lucro_liquido,8538000.0
19119,33.000.167/0001-01,2020-12-31,lucro_liquido,60452000.0


### Quantidade de Ações

In [285]:
df_qtd_acoes = pd.read_csv('../cvm_data/fre/consolidated/fre_capital_social_2011_2021.csv', index_col=0)
df_qtd_acoes['Data_Referencia'] = pd.to_datetime(df_qtd_acoes['Data_Referencia'])
df_qtd_acoes = df_qtd_acoes[df_qtd_acoes['CNPJ_Companhia'] == CNPJ]
df_qtd_acoes = df_qtd_acoes[df_qtd_acoes['Tipo_Capital'] == 'Capital Integralizado']
df_qtd_acoes = df_qtd_acoes[['CNPJ_Companhia', 'Data_Referencia', 'Quantidade_Total_Acoes']]

df_qtd_acoes.columns = ['cnpj', 'dt_refer', 'value']
df_qtd_acoes.insert(2, 'description', 'qtd_acoes')
df_qtd_acoes

,cnpj,dt_refer,description,value
1514,33.000.167/0001-01,2011-01-01,qtd_acoes,13044496930
1517,33.000.167/0001-01,2011-01-01,qtd_acoes,13044496930
1756,33.000.167/0001-01,2012-01-01,qtd_acoes,13044496930
1759,33.000.167/0001-01,2012-01-01,qtd_acoes,13044496930
1914,33.000.167/0001-01,2013-01-01,qtd_acoes,13044496930
1917,33.000.167/0001-01,2013-01-01,qtd_acoes,13044496930
1920,33.000.167/0001-01,2013-01-01,qtd_acoes,13044496930
1916,33.000.167/0001-01,2014-01-01,qtd_acoes,13044496930
1919,33.000.167/0001-01,2014-01-01,qtd_acoes,13044496930
1922,33.000.167/0001-01,2014-01-01,qtd_acoes,13044496930


In [299]:
# df_merge = pd.concat([df_petr_lucro_liquido, df_qtd_acoes])
# df_merge = df_merge.pivot_table(
#     index=['cnpj', 'dt_refer'],
#     columns='description',
#     values='value'
# )

# df_merge['qtd_acoes'] = df_merge['qtd_acoes'].fillna(method='ffill')
# df_merge = df_merge.dropna(subset=['lucro_liquido'])
# df_merge['lucro_liquido'] = df_merge['lucro_liquido'].rolling(4).sum()

# df_merge['LPA'] = df_merge['lucro_liquido'] * 1000 / df_merge['qtd_acoes']

# df_merge

### Balanço Patrimonial - Passivo

In [301]:
def preprocess_bpp(codigo_cvm):

    df_dfp = pd.read_csv('..\cvm_data\data\DFP_BPP_con_2011_2021.csv', index_col=0)
    df_itr = pd.read_csv('..\cvm_data\data\ITR_BPP_con_2011_2021.csv', index_col=0)

    df = pd.concat([df_dfp, df_itr])

    df = df[df['CD_CVM'] == COD_CVM]
    df = df[df['ORDEM_EXERC'] == 'ÚLTIMO']

    df['DT_REFER'] = pd.to_datetime(df['DT_REFER'])
    df['DT_FIM_EXERC'] = pd.to_datetime(df['DT_FIM_EXERC'])



    return df


df_bpp = preprocess_bpp(COD_CVM)

df_patrimonio_liquido = df_bpp[
    (df_bpp['DS_CONTA'].str.contains('Patrimônio Líquido'))
]
df_patrimonio_liquido = df_patrimonio_liquido[['CNPJ_CIA', 'DT_REFER', 'VL_CONTA']]
df_patrimonio_liquido.columns = ['cnpj', 'dt_refer', 'value']
df_patrimonio_liquido.insert(2, 'description', 'patrimonio_liquido')
df_patrimonio_liquido

C:\Users\edyalenquer\AppData\Roaming\Python\Python37\site-packages\numpy\lib\arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,cnpj,dt_refer,description,value
46245,33.000.167/0001-01,2011-12-31,patrimonio_liquido,332223611.0
45356,33.000.167/0001-01,2012-12-31,patrimonio_liquido,345433653.0
44683,33.000.167/0001-01,2013-12-31,patrimonio_liquido,349333684.0
43338,33.000.167/0001-01,2014-12-31,patrimonio_liquido,310722000.0
42769,33.000.167/0001-01,2015-12-31,patrimonio_liquido,257930000.0
42736,33.000.167/0001-01,2016-12-31,patrimonio_liquido,252743000.0
42135,33.000.167/0001-01,2017-12-31,patrimonio_liquido,269609000.0
42896,33.000.167/0001-01,2018-12-31,patrimonio_liquido,283543000.0
51455,33.000.167/0001-01,2019-12-31,patrimonio_liquido,299137000.0
61151,33.000.167/0001-01,2020-12-31,patrimonio_liquido,311150000.0


In [296]:
df_dfp = pd.read_csv('..\cvm_data\data\DFP_BPP_con_2011_2021.csv', index_col=0)
df_itr = pd.read_csv('..\cvm_data\data\ITR_BPP_con_2011_2021.csv', index_col=0)

df = pd.concat([df_dfp, df_itr])
df.head()

C:\Users\edyalenquer\AppData\Roaming\Python\Python37\site-packages\numpy\lib\arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,CNPJ_CIA,DT_REFER,VERSAO,DENOM_CIA,CD_CVM,GRUPO_DFP,MOEDA,ESCALA_MOEDA,ORDEM_EXERC,DT_FIM_EXERC,CD_CONTA,DS_CONTA,VL_CONTA,ST_CONTA_FIXA
0,00.000.000/0001-91,2011-12-31,4,BCO BRASIL S.A.,1023,DF Consolidado - Balanço Patrimonial Passivo,REAL,MIL,PENÚLTIMO,2010-12-31,2,Passivo Total,802819795.0,S
1,00.000.000/0001-91,2011-12-31,4,BCO BRASIL S.A.,1023,DF Consolidado - Balanço Patrimonial Passivo,REAL,MIL,ÚLTIMO,2011-12-31,2,Passivo Total,966823068.0,S
2,00.000.000/0001-91,2011-12-31,4,BCO BRASIL S.A.,1023,DF Consolidado - Balanço Patrimonial Passivo,REAL,MIL,PENÚLTIMO,2010-12-31,2.01,Passivos Financeiros para Negociação,0.0,S
3,00.000.000/0001-91,2011-12-31,4,BCO BRASIL S.A.,1023,DF Consolidado - Balanço Patrimonial Passivo,REAL,MIL,ÚLTIMO,2011-12-31,2.01,Passivos Financeiros para Negociação,0.0,S
4,00.000.000/0001-91,2011-12-31,4,BCO BRASIL S.A.,1023,DF Consolidado - Balanço Patrimonial Passivo,REAL,MIL,PENÚLTIMO,2010-12-31,2.02,Outros Passivos Financeiros ao Valor Justo no ...,5202303.0,S


In [297]:
# CODIGO_CVM = 9512 # PETROBRAS
# IS_ON = False

df_petr = df[df['CD_CVM'] == COD_CVM]
df_petr = df_petr[df_petr['ORDEM_EXERC'] == 'ÚLTIMO']

df_petr['DT_REFER'] = pd.to_datetime(df_petr['DT_REFER'])
# df_petr['DT_INI_EXERC'] = pd.to_datetime(df_petr['DT_INI_EXERC'])
df_petr['DT_FIM_EXERC'] = pd.to_datetime(df_petr['DT_FIM_EXERC'])

display(df_petr.head())

,CNPJ_CIA,DT_REFER,VERSAO,DENOM_CIA,CD_CVM,GRUPO_DFP,MOEDA,ESCALA_MOEDA,ORDEM_EXERC,DT_FIM_EXERC,CD_CONTA,DS_CONTA,VL_CONTA,ST_CONTA_FIXA
46073,33.000.167/0001-01,2011-12-31,2,PETROLEO BRASILEIRO S.A. PETROBRAS,9512,DF Consolidado - Balanço Patrimonial Passivo,REAL,MIL,ÚLTIMO,2011-12-31,2,Passivo Total,599149983.0,S
46075,33.000.167/0001-01,2011-12-31,2,PETROLEO BRASILEIRO S.A. PETROBRAS,9512,DF Consolidado - Balanço Patrimonial Passivo,REAL,MIL,ÚLTIMO,2011-12-31,2.01,Passivo Circulante,68212334.0,S
46077,33.000.167/0001-01,2011-12-31,2,PETROLEO BRASILEIRO S.A. PETROBRAS,9512,DF Consolidado - Balanço Patrimonial Passivo,REAL,MIL,ÚLTIMO,2011-12-31,2.01.01,Obrigações Sociais e Trabalhistas,3182067.0,S
46079,33.000.167/0001-01,2011-12-31,2,PETROLEO BRASILEIRO S.A. PETROBRAS,9512,DF Consolidado - Balanço Patrimonial Passivo,REAL,MIL,ÚLTIMO,2011-12-31,2.01.01.01,Obrigações Sociais,436481.0,S
46081,33.000.167/0001-01,2011-12-31,2,PETROLEO BRASILEIRO S.A. PETROBRAS,9512,DF Consolidado - Balanço Patrimonial Passivo,REAL,MIL,ÚLTIMO,2011-12-31,2.01.01.02,Obrigações Trabalhistas,2745586.0,S


### Trata Patrimônio Líquido

In [298]:
year = 2021

df_temp = df_petr[
    (df_petr['DS_CONTA'].str.contains('Patrimônio Líquido')) &
    (df_petr['DT_REFER'].dt.year == year)
]
df_temp

,CNPJ_CIA,DT_REFER,VERSAO,DENOM_CIA,CD_CVM,GRUPO_DFP,MOEDA,ESCALA_MOEDA,ORDEM_EXERC,DT_FIM_EXERC,CD_CONTA,DS_CONTA,VL_CONTA,ST_CONTA_FIXA
61363,33.000.167/0001-01,2021-12-31,1,PETROLEO BRASILEIRO S.A. PETROBRAS,9512,DF Consolidado - Balanço Patrimonial Passivo,REAL,MIL,ÚLTIMO,2021-12-31,2.03,Patrimônio Líquido Consolidado,389581000.0,S
183555,33.000.167/0001-01,2021-03-31,1,PETROLEO BRASILEIRO S.A. PETROBRAS,9512,DF Consolidado - Balanço Patrimonial Passivo,REAL,MIL,ÚLTIMO,2021-03-31,2.03,Patrimônio Líquido Consolidado,320245000.0,S
183773,33.000.167/0001-01,2021-06-30,1,PETROLEO BRASILEIRO S.A. PETROBRAS,9512,DF Consolidado - Balanço Patrimonial Passivo,REAL,MIL,ÚLTIMO,2021-06-30,2.03,Patrimônio Líquido Consolidado,362390000.0,S
183991,33.000.167/0001-01,2021-09-30,1,PETROLEO BRASILEIRO S.A. PETROBRAS,9512,DF Consolidado - Balanço Patrimonial Passivo,REAL,MIL,ÚLTIMO,2021-09-30,2.03,Patrimônio Líquido Consolidado,375689000.0,S


In [145]:
df_petr[
    (df_petr['DS_CONTA'].str.contains('Lucro')) &
    (df_petr['DS_CONTA'].str.contains('Prejuízo')) &
    (df_petr['DS_CONTA'].str.contains('Consolidado')) &
    (df_petr['DT_REFER'].dt.year == 2019)
].sort_values('DT_REFER')

,CNPJ_CIA,DT_REFER,VERSAO,DENOM_CIA,CD_CVM,GRUPO_DFP,MOEDA,ESCALA_MOEDA,ORDEM_EXERC,DT_INI_EXERC,TIPO_PERIODO,DT_FIM_EXERC,CD_CONTA,DS_CONTA,VL_CONTA,ST_CONTA_FIXA
65393,33.000.167/0001-01,2019-03-31,1,PETROLEO BRASILEIRO S.A. PETROBRAS,9512,DF Consolidado - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2019-01-01,TRIMESTRAL,2019-03-31,3.11,Lucro/Prejuízo Consolidado do Período,4240000.0,S
65547,33.000.167/0001-01,2019-06-30,1,PETROLEO BRASILEIRO S.A. PETROBRAS,9512,DF Consolidado - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2019-04-01,TRIMESTRAL,2019-06-30,3.11,Lucro/Prejuízo Consolidado do Período,19352000.0,S
65719,33.000.167/0001-01,2019-09-30,1,PETROLEO BRASILEIRO S.A. PETROBRAS,9512,DF Consolidado - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2019-07-01,TRIMESTRAL,2019-09-30,3.11,Lucro/Prejuízo Consolidado do Período,8840000.0,S
16367,33.000.167/0001-01,2019-12-31,1,PETROLEO BRASILEIRO S.A. PETROBRAS,9512,DF Consolidado - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2019-01-01,TRIMESTRAL,2019-12-31,3.11,Lucro/Prejuízo Consolidado do Período,8538000.0,S


In [73]:
DATA_PATH = os.path.join('..', 'cvm_data', 'data')

for document_type in document_types:
    print(document_type, '\n')

    data = pd.read_csv(os.path.join(DATA_PATH, f'{document_type}_2011_2021.csv'))

    print(data['DS_CONTA'].value_counts().head(10))
    print(data['DT_FIM_EXERC'].value_counts().head(10))
    print('\n\n')

BPA_con 

Outras Contas a Receber                                 46790
Despesas Antecipadas                                    45780
Clientes                                                45575
Ativos de Operações Descontinuadas                      45202
Aplicações Financeiras Avaliadas ao Custo Amortizado    45148
Contas a Receber                                        45054
Estoques                                                45001
Ativos Não-Correntes a Venda                            44542
Ativos Biológicos                                       44534
Títulos para Negociação                                 37587
Name: DS_CONTA, dtype: int64
2020-12-31    84695
2019-12-31    71037
2010-12-31    66308
2011-12-31    64754
2013-12-31    64434
2012-12-31    63942
2016-12-31    63528
2014-12-31    63438
2015-12-31    62406
2018-12-31    61433
Name: DT_FIM_EXERC, dtype: int64



BPA_ind 

Despesas Antecipadas                                    83890
Outras Contas a Receber          

### PETR

In [63]:
CODIGO_CVM = 9512 # PETROBRAS
IS_ON = False


df_dre = pd.read_csv(os.path.join(DATA_PATH, 'DRE_ind_2011_2021.csv'))
df_petr = df_dre[df_dre['CD_CVM'] == CODIGO_CVM]
display(df_petr.head())

,Unnamed: 0,CNPJ_CIA,DT_REFER,VERSAO,DENOM_CIA,CD_CVM,GRUPO_DFP,MOEDA,ESCALA_MOEDA,ORDEM_EXERC,DT_INI_EXERC,DT_FIM_EXERC,CD_CONTA,DS_CONTA,VL_CONTA,ST_CONTA_FIXA
136088,136088,33.000.167/0001-01,2011-03-31,1,PETROLEO BRASILEIRO S.A. PETROBRAS,9512,DF Individual - Demonstração do Resultado,REAL,MIL,PENÚLTIMO,2010-01-01,2010-03-31,3.01,Receita de Venda de Bens e/ou Serviços,36951907.0,S
136089,136089,33.000.167/0001-01,2011-03-31,1,PETROLEO BRASILEIRO S.A. PETROBRAS,9512,DF Individual - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2011-01-01,2011-03-31,3.01,Receita de Venda de Bens e/ou Serviços,40096584.0,S
136090,136090,33.000.167/0001-01,2011-03-31,1,PETROLEO BRASILEIRO S.A. PETROBRAS,9512,DF Individual - Demonstração do Resultado,REAL,MIL,PENÚLTIMO,2010-01-01,2010-03-31,3.02,Custo dos Bens e/ou Serviços Vendidos,-21342361.0,S
136091,136091,33.000.167/0001-01,2011-03-31,1,PETROLEO BRASILEIRO S.A. PETROBRAS,9512,DF Individual - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2011-01-01,2011-03-31,3.02,Custo dos Bens e/ou Serviços Vendidos,-24702863.0,S
136092,136092,33.000.167/0001-01,2011-03-31,1,PETROLEO BRASILEIRO S.A. PETROBRAS,9512,DF Individual - Demonstração do Resultado,REAL,MIL,PENÚLTIMO,2010-01-01,2010-03-31,3.03,Resultado Bruto,15609546.0,S


In [64]:
df_petr = df_petr[df_petr['ORDEM_EXERC']=='ÚLTIMO']

In [71]:
df_petr.sort_values('DT_INI_EXERC').head(50)

,Unnamed: 0,CNPJ_CIA,DT_REFER,VERSAO,DENOM_CIA,CD_CVM,GRUPO_DFP,MOEDA,ESCALA_MOEDA,ORDEM_EXERC,DT_INI_EXERC,DT_FIM_EXERC,CD_CONTA,DS_CONTA,VL_CONTA,ST_CONTA_FIXA
136089,136089,33.000.167/0001-01,2011-03-31,1,PETROLEO BRASILEIRO S.A. PETROBRAS,9512,DF Individual - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2011-01-01,2011-03-31,3.01,Receita de Venda de Bens e/ou Serviços,4.009658e+07,S
136330,136330,33.000.167/0001-01,2011-09-30,2,PETROLEO BRASILEIRO S.A. PETROBRAS,9512,DF Individual - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2011-01-01,2011-09-30,3.04,Despesas/Receitas Operacionais,-1.529230e+07,S
136326,136326,33.000.167/0001-01,2011-09-30,2,PETROLEO BRASILEIRO S.A. PETROBRAS,9512,DF Individual - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2011-01-01,2011-09-30,3.03,Resultado Bruto,4.686084e+07,S
136466,136466,33.000.167/0001-01,2011-09-30,2,PETROLEO BRASILEIRO S.A. PETROBRAS,9512,DF Individual - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2011-01-01,2011-09-30,3.99.02.02,PN,2.151980e+00,N
136322,136322,33.000.167/0001-01,2011-09-30,2,PETROLEO BRASILEIRO S.A. PETROBRAS,9512,DF Individual - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2011-01-01,2011-09-30,3.02,Custo dos Bens e/ou Serviços Vendidos,-8.716674e+07,S
136318,136318,33.000.167/0001-01,2011-09-30,2,PETROLEO BRASILEIRO S.A. PETROBRAS,9512,DF Individual - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2011-01-01,2011-09-30,3.01,Receita de Venda de Bens e/ou Serviços,1.340276e+08,S
136314,136314,33.000.167/0001-01,2011-06-30,1,PETROLEO BRASILEIRO S.A. PETROBRAS,9512,DF Individual - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2011-01-01,2011-06-30,3.99.02.02,PN,1.667790e+00,N
136310,136310,33.000.167/0001-01,2011-06-30,1,PETROLEO BRASILEIRO S.A. PETROBRAS,9512,DF Individual - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2011-01-01,2011-06-30,3.99.02.01,ON,1.667790e+00,N
136306,136306,33.000.167/0001-01,2011-06-30,1,PETROLEO BRASILEIRO S.A. PETROBRAS,9512,DF Individual - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2011-01-01,2011-06-30,3.99.02,Lucro Diluído por Ação,0.000000e+00,N
136302,136302,33.000.167/0001-01,2011-06-30,1,PETROLEO BRASILEIRO S.A. PETROBRAS,9512,DF Individual - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2011-01-01,2011-06-30,3.99.01.02,PN,1.667790e+00,N


In [70]:
df_petr[
    # (df_petr['DS_CONTA'].str.contains('Lucro')) &
    # (df_petr['DS_CONTA'].str.contains('Prejuízo'))
    # (df_petr['DS_CONTA'].str.contains('Período'))
].sort_values('DT_INI_EXERC').head(50)

SyntaxError: invalid syntax (4272614405.py, line 5)